# Capstone Project Assignment - Part 1


Importing relevant libraries

In [5]:
import pandas as pd
import numpy as np
import folium
import geopy
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
%matplotlib inline 
from sklearn.cluster import KMeans 

In [4]:
pip install geocoder

     |████████████████████████████████| 98 kB 8.5 MB/s  eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


### Download data from wiki and assign to pandas dataframe using Pandas read_html function

In [6]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### Shape of Actual Dataframe

In [7]:
print("Actual shape of Frame",df.shape)

Actual shape of Frame (180, 3)


### Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned

In [8]:
df.drop(df[df['Borough'] == 'Not assigned'].index, inplace = True) 

print("Shape of Frame after filtering ",df.shape)

Shape of Frame after filtering  (103, 3)


### More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma
#### -- Dataframe Already have the neighbour combined per postal code

In [9]:
df[df["Postal Code"]=="M5A"]

,Postal Code,Borough,Neighbourhood
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### Check if any Postal Code have duplicate entry in dataframe

In [10]:
df['Postal Code'].duplicated().any()

False

### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

In [11]:
df["Neighbourhood"].loc[df["Neighbourhood"]=="Not assigned"] = df["Borough"]

### Checking if any row exists with Neighbourhood = "Not assigned"

In [12]:
df[df["Neighbourhood"]=="Not assigned"].count()

Postal Code      0
Borough          0
Neighbourhood    0
dtype: int64

### Final shape of Dataframe after Cleanup

In [13]:
print("Final shape",df.shape)

Final shape (103, 3)


### Reset Index of Dataframe after filtering


In [14]:
df.reset_index(inplace=True, drop=True)
df.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### Using the csv link to populate latitude and longitude for postal codes

In [15]:
import requests
import io
datastr = requests.get("http://cocl.us/Geospatial_data",allow_redirects=True).text
data_file = io.StringIO(datastr)
geo=pd.read_csv(data_file)
geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Creating combined Dataframe using df and geo

In [16]:
df_geo=pd.merge(df, geo, on='Postal Code')
df_geo.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


## Creating new dataset with all the postal code with Borough having "Toronto"

In [17]:
df_toronto=df_geo[df_geo['Borough'].str.contains("Toronto")].reset_index(drop=True)

In [18]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_toronto['Borough'].unique()),
        df_toronto.shape[0]
    )
)

The dataframe has 5 boroughs and 40 neighborhoods.


### Getting Latitude and longitude for Toronto using geopy

In [19]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


## Plotting the neighborhood of Toronto in map

In [20]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Defining Four square API credentials to explore each borough in dataframe

In [2]:
{
    "tags": [
        "hide-input",
    ]
}



### Let's explore the first neighborhood in our dataframe.

In [23]:
df_toronto.loc[0, 'Neighbourhood']

'Regent Park, Harbourfront'

Get the neighborhood's latitude and longitude values.

In [24]:
neighborhood_latitude = df_toronto.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_toronto.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_toronto.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.6542599, -79.3606359.


### let's get the top 100 venues that are in Regent Park, Harbourfront within a radius of 500 meters.

In [25]:
radius=500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET,VERSION, latitude, longitude, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=2ZMEVFJ5N5HDJSESYXN4PN4ZDEQS0WACSVE1H2XWDEQHPBS4&client_secret=2LAVDGJINGXBOSBEIUMHR5SLVJSY3Z3NMADVGVLSQQCPT4KU&v=20180605&ll=43.6534817,-79.3839347&radius=500&limit=100'

Send get request to retrieve the data from foursquare

In [29]:
import json
from pandas.io.json import json_normalize

In [26]:
results = requests.get(url).json()

In [27]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [30]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,name,categories,lat,lng
0,Downtown Toronto,Neighborhood,43.653232,-79.385296
1,Nathan Phillips Square,Plaza,43.652270,-79.383516
2,Japango,Sushi Restaurant,43.655268,-79.385165
3,Indigo,Bookstore,43.653515,-79.380696
4,Poke Guys,Poke Place,43.654895,-79.385052


In [31]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

73 venues were returned by Foursquare.


## Explore Neighborhoods in Toronto

Creating a function to repeat the same process to all neighborhood of Toronto

In [32]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [33]:
Toronto_venues = getNearbyVenues(names=df_toronto['Neighbourhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Runnymede, The Junction, Weston-Pellam Park, Carlton Village
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harb

Shape of Venue dataset

In [34]:
print(Toronto_venues.shape)
Toronto_venues.head()

(1614, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


List the number of venues retrurned by each neighbourhood

In [35]:
Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,59,59,59,59,59,59
"Brockton, Parkdale Village, Exhibition Place",22,22,22,22,22,22
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",16,16,16,16,16,16
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",17,17,17,17,17,17
Central Bay Street,61,61,61,61,61,61
Christie,16,16,16,16,16,16
Church and Wellesley,78,78,78,78,78,78
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,36,36,36,36,36,36


Figuring out unique venue category

In [36]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 235 uniques categories.


## Analysing Neighborhood

In [37]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 


# move neighborhood column to the first column
first_col = Toronto_onehot['Neighborhood']
Toronto_onehot.drop(labels=['Neighborhood'], axis=1, inplace = True)
Toronto_onehot.insert(0, 'Neighborhood', first_col)


Toronto_onehot.head()

,Neighborhood,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,...,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Checking Shape

In [38]:
Toronto_onehot.shape

(1614, 235)

### let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [39]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped.head()

,Neighborhood,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,...,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.016949,...,0.0,0.0,0.0,0.0,0.0,0.016949,0.0,0.0,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000
2,"Business reply mail Processing Centre, South C...",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.062500
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.058824,0.058824,0.117647,0.176471,0.117647,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000
4,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.016393,0.0,0.0,0.016393,0.016393


Checking shape of the grouped dataset

In [40]:
Toronto_grouped.shape

(40, 235)

### Let's print each neighborhood along with the top 5 most common venues

In [41]:
num_top_venues = 5

for hood in Toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
            venue  freq
0     Coffee Shop  0.08
1    Cocktail Bar  0.05
2          Bakery  0.05
3        Pharmacy  0.03
4  Farmers Market  0.03


----Brockton, Parkdale Village, Exhibition Place----
                venue  freq
0                Café  0.14
1      Breakfast Spot  0.09
2         Coffee Shop  0.09
3   Convenience Store  0.05
4  Italian Restaurant  0.05


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
           venue  freq
0    Yoga Studio  0.06
1  Garden Center  0.06
2    Pizza Place  0.06
3     Comic Shop  0.06
4     Restaurant  0.06


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0   Airport Service  0.18
1    Airport Lounge  0.12
2  Airport Terminal  0.12
3           Airport  0.06
4               Bar  0.06


----Central Bay Street----
                 venue  freq
0          Coffee Shop  0.18
1   Italian Restaura

#### function to sort the venues in descending order.

In [42]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### creating the new dataframe and display the top 10 venues for each neighborhood.

In [43]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Pharmacy,Farmers Market,Beer Bar,Restaurant,Cheese Shop,Seafood Restaurant,Pub
1,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Coffee Shop,Convenience Store,Italian Restaurant,Pet Store,Stadium,Furniture / Home Store,Climbing Gym,Nightclub
2,"Business reply mail Processing Centre, South C...",Yoga Studio,Garden Center,Pizza Place,Comic Shop,Restaurant,Farmers Market,Fast Food Restaurant,Skate Park,Burrito Place,Smoke Shop
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Airport,Bar,Rental Car Location,Coffee Shop,Sculpture Garden,Boutique,Boat or Ferry
4,Central Bay Street,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Japanese Restaurant,Salad Place,Burger Joint,Thai Restaurant,Bubble Tea Shop,Dessert Shop


## Cluster Neighborhoods

Run k-means to cluster the neighborhood into 5 clusters.

In [108]:
# set number of clusters

kclusters = 7

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:20] 

array([6, 6, 6, 6, 0, 3, 6, 6, 6, 6, 6, 6, 4, 6, 6, 6, 6, 6, 5, 6],
      dtype=int32)

In [109]:
#df_toronto.rename(columns = {'Neighbourhood':'Neighborhood'}, inplace = True)
#df_toronto.head()
neighborhoods_venues_sorted.drop(['Cluster Labels'], axis=1, inplace=True)

creating a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [110]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_toronto

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,6,Coffee Shop,Park,Bakery,Theater,Café,Restaurant,Pub,Breakfast Spot,Gym / Fitness Center,Event Space
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Diner,Sushi Restaurant,Yoga Studio,Japanese Restaurant,Fried Chicken Joint,Sandwich Place,Beer Bar,Bar,Bank
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,6,Clothing Store,Coffee Shop,Cosmetics Shop,Hotel,Italian Restaurant,Japanese Restaurant,Café,Bubble Tea Shop,Middle Eastern Restaurant,Pizza Place
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,6,Coffee Shop,Café,Gastropub,Cocktail Bar,Lingerie Store,Clothing Store,Seafood Restaurant,Restaurant,Moroccan Restaurant,Cosmetics Shop
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,4,Health Food Store,Trail,Pub,Airport,Movie Theater,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant


In [111]:
toronto_merged['Cluster Labels'].value_counts()

6    29
0     3
1     2
3     2
4     2
2     1
5     1
Name: Cluster Labels, dtype: int64

### Finally, let's visualize the resulting clusters

In [112]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Cluster

### First Cluster

In [113]:
toronto_merged.loc[toronto_merged['Cluster Labels'] ==0, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,"Queen's Park, Ontario Provincial Government",0,Coffee Shop,Diner,Sushi Restaurant,Yoga Studio,Japanese Restaurant,Fried Chicken Joint,Sandwich Place,Beer Bar,Bar,Bank
6,Central Bay Street,0,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Japanese Restaurant,Salad Place,Burger Joint,Thai Restaurant,Bubble Tea Shop,Dessert Shop
25,"The Annex, North Midtown, Yorkville",0,Sandwich Place,Café,Coffee Shop,History Museum,Furniture / Home Store,Pizza Place,Park,Pub,Donut Shop,Middle Eastern Restaurant


### Second Cluster

In [114]:
toronto_merged.loc[toronto_merged['Cluster Labels'] ==1, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
30,"Moore Park, Summerhill East",1,Park,Airport,Movie Theater,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant
34,Rosedale,1,Park,Playground,Trail,Airport,Moroccan Restaurant,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store


### 

### Third Cluster

In [116]:
toronto_merged.loc[toronto_merged['Cluster Labels'] ==2, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Roselawn,2,Home Service,Pool,Garden,Airport,Museum,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant


### Fourth Cluster

In [117]:
toronto_merged.loc[toronto_merged['Cluster Labels'] ==3, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Christie,3,Grocery Store,Café,Park,Coffee Shop,Nightclub,Baby Store,Candy Store,Athletics & Sports,Restaurant,Italian Restaurant
20,"Runnymede, The Junction, Weston-Pellam Park, C...",3,Breakfast Spot,Grocery Store,Bus Line,Caribbean Restaurant,Airport,Museum,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant


### Fifth Cluster

In [118]:
toronto_merged.loc[toronto_merged['Cluster Labels'] ==4, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,The Beaches,4,Health Food Store,Trail,Pub,Airport,Movie Theater,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant
22,"Forest Hill North & West, Forest Hill Road Park",4,Trail,Jewelry Store,Mexican Restaurant,Sushi Restaurant,Airport,Moroccan Restaurant,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant


### Sixth Cluster

In [119]:
toronto_merged.loc[toronto_merged['Cluster Labels'] ==5, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Lawrence Park,5,Park,Bus Line,Business Service,Swim School,Airport,Moroccan Restaurant,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant


### Seventh Cluster

In [120]:
toronto_merged.loc[toronto_merged['Cluster Labels'] ==6, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Regent Park, Harbourfront",6,Coffee Shop,Park,Bakery,Theater,Café,Restaurant,Pub,Breakfast Spot,Gym / Fitness Center,Event Space
2,"Garden District, Ryerson",6,Clothing Store,Coffee Shop,Cosmetics Shop,Hotel,Italian Restaurant,Japanese Restaurant,Café,Bubble Tea Shop,Middle Eastern Restaurant,Pizza Place
3,St. James Town,6,Coffee Shop,Café,Gastropub,Cocktail Bar,Lingerie Store,Clothing Store,Seafood Restaurant,Restaurant,Moroccan Restaurant,Cosmetics Shop
5,Berczy Park,6,Coffee Shop,Cocktail Bar,Bakery,Pharmacy,Farmers Market,Beer Bar,Restaurant,Cheese Shop,Seafood Restaurant,Pub
8,"Richmond, Adelaide, King",6,Coffee Shop,Café,Restaurant,Clothing Store,Bakery,Deli / Bodega,Gym,Thai Restaurant,Hotel,Steakhouse
9,"Dufferin, Dovercourt Village",6,Pharmacy,Bakery,Athletics & Sports,Park,Music Venue,Café,Middle Eastern Restaurant,Bus Stop,Brewery,Supermarket
10,"Harbourfront East, Union Station, Toronto Islands",6,Coffee Shop,Aquarium,Hotel,Café,Fried Chicken Joint,Restaurant,Brewery,Scenic Lookout,Italian Restaurant,Baseball Stadium
11,"Little Portugal, Trinity",6,Bar,Vegetarian / Vegan Restaurant,Men's Store,Asian Restaurant,Café,Restaurant,French Restaurant,Record Shop,Cocktail Bar,Juice Bar
12,"The Danforth West, Riverdale",6,Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Ice Cream Shop,Lounge,Fruit & Vegetable Store,Frozen Yogurt Shop,Bookstore,Liquor Store
13,"Toronto Dominion Centre, Design Exchange",6,Coffee Shop,Hotel,Café,Restaurant,Seafood Restaurant,Salad Place,Bakery,Italian Restaurant,Japanese Restaurant,Sporting Goods Shop
